In [1]:
import argparse
import math
import os

import pandas as pd
import torch
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from torch import nn
from torch.utils.data import DataLoader

from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import numpy as np
from datasets import get_train_test_data2, MMoE_Dataset,get_train_test_data1
from models import MMoE_Model, Transfer_Model, MMoE_Model_Expert


In [2]:
file_path = "./data/new_data/process_data/mutilsolvent2.csv"

In [3]:
data = pd.read_csv(file_path).values

In [4]:
from sklearn.model_selection import train_test_split


def get_train_test_data(file_path, test_size,seed,dataset_num):
    data = pd.read_csv(file_path).values
    x_train_list=[]
    y_train_list=[]
    x_test_list=[]
    y_test_list=[]
    for i in range(15):
        if i == dataset_num:
            cur_data = data[35*i:35*(i+1),:]
            transfer_x,transfer_y = cur_data[:, :-4], cur_data[:, -4:]
        else:
            cur_data = data[35*i:35*(i+1),:]
            X, y = cur_data[:, :-4], cur_data[:, -4:]
            x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test_size,random_state=seed)
            x_train_list.append(x_train)
            y_train_list.append(y_train)
            x_test_list.append(x_test)
            y_test_list.append(y_test)
    x_train, x_test, y_train, y_test = np.concatenate(x_train_list),np.concatenate(x_test_list),np.concatenate(y_train_list),np.concatenate(y_test_list)
    print(f"dataset num {dataset_num}")
    print(f"the number of train samples is: {len(x_train)}")
    print(f"the number of test samples is: {len(x_test)}")
    print(f"the number of transfer samples is: {len(x_test)}")
    return x_train, x_test, y_train, y_test,transfer_x,transfer_y


In [5]:
for i in range(15):
    x_train, x_test, y_train, y_test,transfer_X,transfer_y = get_train_test_data(file_path,0.2,42,i)

dataset num 0
the number of train samples is: 392
the number of test samples is: 98
the number of transfer samples is: 98
dataset num 1
the number of train samples is: 392
the number of test samples is: 98
the number of transfer samples is: 98
dataset num 2
the number of train samples is: 392
the number of test samples is: 98
the number of transfer samples is: 98
dataset num 3
the number of train samples is: 392
the number of test samples is: 98
the number of transfer samples is: 98
dataset num 4
the number of train samples is: 392
the number of test samples is: 98
the number of transfer samples is: 98
dataset num 5
the number of train samples is: 392
the number of test samples is: 98
the number of transfer samples is: 98
dataset num 6
the number of train samples is: 392
the number of test samples is: 98
the number of transfer samples is: 98
dataset num 7
the number of train samples is: 392
the number of test samples is: 98
the number of transfer samples is: 98
dataset num 8
the number

In [6]:
def create_dir(file_path):
    if not os.path.exists(os.path.dirname(file_path)):
        os.makedirs(os.path.dirname(file_path))


def trainer(water_model,water_select_expert,air_model,air_select_expert,train_loader, model, model_save_path, device ,lr, epochs, early_stop_num, verbose=True, writer_flag=False):
    model = model.to(device)
    criterion = nn.MSELoss(reduction='mean')
    params = model.parameters()
    theta1 = model.theta1
    theta2 = model.theta2
    theta3 = model.theta3
    theta4 = model.theta4

    water_model=water_model.to(device)
    air_model=air_model.to(device)
    
    optimizer = torch.optim.Adam(params, lr=lr)

    create_dir(model_save_path)

    n_epochs, best_loss, step, early_stop_count = epochs, math.inf, 0, 0


    best_loss1 = best_loss2 = best_loss3 = best_loss4 = math.inf
    writer = None
    if writer_flag:
        writer = SummaryWriter()
    ran = range(n_epochs)
    if not verbose:
        ran = tqdm(range(n_epochs), position=0, leave=True)
    for epoch in ran:
        model.train()
        loss_total_record = []
        loss1_record = []
        loss2_record = []
        loss3_record = []
        loss4_record = []

        for x, y1, y2, y3, y4 in train_loader:

            x, y1, y2, y3, y4 = x.to(device), y1.to(device), y2.to(device), y3.to(device), y4.to(device),


            water_expert_represents = water_model(x)[:, water_select_expert, :]
            air_expert_represents = air_model(x)[:, air_select_expert, :]
            
            
            pred1, pred2, pred3, pred4 = model(x,water_expert_represents,air_expert_represents)

            loss1 = criterion(pred1, y1)
            loss2 = criterion(pred2, y2)
            loss3 = criterion(pred3, y3)
            loss4 = criterion(pred4, y4)


            loss_total = loss1 / (theta1 ** 2) +  loss2 / (theta2 ** 2) + loss3 / (theta3 ** 2)*1000 + loss4 / (theta4 ** 2) + 2 * (torch.log(theta1) +torch.log(theta2) + torch.log(theta3) + torch.log(theta4))
            optimizer.zero_grad()
            loss_total.backward()
            optimizer.step()
            step += 1

            loss_total_record.append(loss_total.detach().item())
            loss1_record.append(loss1.detach().item())
            loss2_record.append(loss2.detach().item())
            loss3_record.append(loss3.detach().item())
            loss4_record.append(loss4.detach().item())

        mean_train_loss_total = sum(loss_total_record) / len(loss_total_record)
        mean_train_loss1 = sum(loss1_record) / len(loss1_record)
        mean_train_loss2 = sum(loss2_record) / len(loss2_record)
        mean_train_loss3 = sum(loss3_record) / len(loss3_record)
        mean_train_loss4 = sum(loss4_record) / len(loss4_record)
        if writer_flag:
            writer.add_scalar('Loss_total/train', mean_train_loss_total, step)
            writer.add_scalar('Loss1/train', mean_train_loss1, step)
            writer.add_scalar('Loss2/train', mean_train_loss2, step)
            writer.add_scalar('Loss3/train', mean_train_loss3, step)
            writer.add_scalar('Loss4/train', mean_train_loss4, step)

        if verbose and epoch % 100 == 99:
            print(
                f'Epoch [{epoch + 1}/{n_epochs}]: Train loss_total: {mean_train_loss_total:.6f}, loss1: {mean_train_loss1:.6f},loss2: {mean_train_loss2:.6f},loss3: {mean_train_loss3:.6f},loss4: {mean_train_loss4:.6f}')

        if mean_train_loss_total < best_loss:
            best_loss = mean_train_loss_total
            best_loss1 = mean_train_loss1
            best_loss2 = mean_train_loss2
            best_loss3 = mean_train_loss3
            best_loss4 = mean_train_loss4

            torch.save(model.state_dict(), model_save_path)  # Save your best model
            if verbose:
                print(
                    f"\nSave with loss_total: {mean_train_loss_total:.6f}, loss1: {mean_train_loss1:.6f},loss2: {mean_train_loss2:.6f},loss3: {mean_train_loss3:.6f},loss4: {mean_train_loss4:.6f}")
            early_stop_count = 0
        else:
            early_stop_count += 1

        if early_stop_count >= early_stop_num:
            print(f'\nModel is not improving, so we halt the training session at epoch: {epoch + 1}.')
            print(
                f"\n Best Model loss_total: {best_loss:.6f}, loss1: {best_loss1:.6f},loss2: {best_loss2:.6f},loss3: {best_loss3:.6f},loss4: {best_loss4:.6f}")
            print(f"\ntheta1:{theta1},theta2:{theta2},theta3:{theta3},theta4:{theta4}")
            return

    print(f'\nTrain all epochs.')
    print(
        f"\n Best Model loss_total: {best_loss:.6f}, loss1: {best_loss1:.6f},loss2: {best_loss2:.6f},loss3: {best_loss3:.6f},loss4: {best_loss4:.6f}")
    print(f"\ntheta1:{theta1},theta2:{theta2},theta3:{theta3},theta4:{theta4}")



def predict(model,device,data,y,water_expert_represents_test,air_expert_represents_test):
    data = torch.Tensor(data).to(device)
    y1 = torch.Tensor(y[:,0]).to(device)
    y2 = torch.Tensor(y[:,1]).to(device)
    y3 = torch.Tensor(y[:,2]).to(device)
    y4 = torch.Tensor(y[:,3]).to(device)

    criterion = nn.MSELoss(reduction="mean")
    model.eval()
    pred1,pred2,pred3,pred4= model(data,water_expert_represents_test,air_expert_represents_test)

    pred_list = [pred1,pred2,pred3,pred4]
    rmse1  = criterion(pred1,y1).item()**0.5
    rmse2  = criterion(pred2,y2).item() ** 0.5
    rmse3  = criterion(pred3,y3).item()** 0.5
    rmse4  = criterion(pred4,y4).item()** 0.5
    rmse_list = [rmse1,rmse2,rmse3,rmse4]

    r1 = r2_score(y1.cpu().detach().numpy(),pred1.cpu().detach().numpy())
    r2 = r2_score(y2.cpu().detach().numpy(),pred2.cpu().detach().numpy())
    r3 = r2_score(y3.cpu().detach().numpy(),pred3.cpu().detach().numpy())
    r4 = r2_score(y4.cpu().detach().numpy(),pred4.cpu().detach().numpy())

    m1 = mean_absolute_error(y1.cpu().detach().numpy(),pred1.cpu().detach().numpy())
    m2 = mean_absolute_error(y2.cpu().detach().numpy(),pred2.cpu().detach().numpy())
    m3 = mean_absolute_error(y3.cpu().detach().numpy(),pred3.cpu().detach().numpy())
    m4 = mean_absolute_error(y4.cpu().detach().numpy(),pred4.cpu().detach().numpy())

    mape1 = mean_absolute_percentage_error(y1.cpu().detach().numpy(),pred1.cpu().detach().numpy())
    mape2 = mean_absolute_percentage_error(y2.cpu().detach().numpy(),pred2.cpu().detach().numpy())
    mape3 = mean_absolute_percentage_error(y3.cpu().detach().numpy(),pred3.cpu().detach().numpy())
    mape4 = mean_absolute_percentage_error(y4.cpu().detach().numpy(),pred4.cpu().detach().numpy())

    r_list = [r1,r2,r3,r4]
    m_list = [m1,m2,m3,m4]
    mape_list = [mape1,mape2,mape3,mape4]
    return rmse_list,r_list,m_list,pred_list,mape_list




##%% md
import numpy as np
##%%
FILE_DIR = "../data/process_data"
MODEL_DIR = "./models"
RESULT_DIR="./results"
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Train')
    parser.add_argument('--file_path', type=str, default='data/new_data/process_data/mutilsolvent2.csv',help='')
    parser.add_argument('--model_save_dir', type=str,default="./p2results_1", help='')
    parser.add_argument('--res_dir', type=str,default="./p2results_1", help='')

    parser.add_argument('--water_model_save_path', type=str,default="water2_model_result/waterdi6zhe6.ckpt", help='')
    parser.add_argument('--air_model_save_path', default="air2_model_result/air2di9zhe6.ckpt",type=str, help='')
    parser.add_argument('--water_select_expert_config', default="water2_res_result/waterdi6zhe6_top2.csv",type=str, help='')
    parser.add_argument('--air_select_expert_config', type=str, default="air2_res_result/air2di9zhe6_top2.csv",help='')

    parser.add_argument('--input_dim', type=int,default=19, help='')
    parser.add_argument('--represent_dim', type=int, default=100, help='')
    parser.add_argument('--pair_embedding_dim', type=int, default=5, help='')
    parser.add_argument('--expert_num', type=int, default=6, help='')
    parser.add_argument('--gate_output_dim', type=int, default=10, help='')
    parser.add_argument('--epochs',  type=int, default=1, help='')
    parser.add_argument('--early_stop_num',  type=int, default=200, help='')

    parser.add_argument('--lr', type=float, default=0.0005, help='')
    parser.add_argument("--verbose", action="store_true", help="")
    parser.add_argument("--writer_flag", action="store_true", help="")
    parser.add_argument('--batch_size',  type=int, default=16, help='')
    parser.add_argument('--test_size', type=float, default=0.2, help='')
    parser.add_argument('--seed',  type=int, default=42, help='')
    parser.add_argument('--eps', type=str, default="", help='')
    parser.add_argument("--frozen", action="store_true", help="")

    # device = "cuda:5" if torch.cuda.is_available() else "cpu"
    device = "cpu"
    karg = parser.parse_args(args=[])
    print(karg)
    file_name = os.path.basename(karg.file_path)
    print(f"\n\n\n====start to train {file_name}====")
    model_save_path =  os.path.join(karg.model_save_dir,file_name.split(".")[0]+".ckpt")
    print(karg.file_path)
    create_dir(model_save_path)

    for k in range(0,15):
        x_train, x_test, y_train, y_test,transfer_X,transfer_y = get_train_test_data(file_path,karg.test_size,karg.seed,k)

        water_model_save_path = karg.water_model_save_path
        air_model_save_path = karg.air_model_save_path
        model_water = MMoE_Model_Expert()
        model_water.load_state_dict(torch.load(water_model_save_path,map_location=torch.device(device)))
        model_air = MMoE_Model_Expert()
        model_air.load_state_dict(torch.load(air_model_save_path,map_location=torch.device(device)))

        if karg.frozen:
            model_water.pair_embedding_layer_list.requires_grad_(False)
            model_water.expert_list.requires_grad_(False)
            model_water.beta_layer.requires_grad_(False)
            model_air.pair_embedding_layer_list.requires_grad_(False)
            model_air.expert_list.requires_grad_(False)
            model_air.beta_layer.requires_grad_(False)



        water_top2_index_df  = pd.read_csv(karg.water_select_expert_config)
        air_top2_index_df = pd.read_csv(karg.air_select_expert_config)

        water_select_expert = torch.Tensor(water_top2_index_df.values).long()
        air_select_expert = torch.Tensor(air_top2_index_df.values).long()




        train_dataset = MMoE_Dataset(x_train, y_train[:, 0], y_train[:, 1], y_train[:, 2], y_train[:, 3])
        train_dataloader = DataLoader(train_dataset, karg.batch_size, shuffle=True, pin_memory=True)
        model = Transfer_Model(input_dim=karg.input_dim, represent_dim=karg.represent_dim, pair_embedding_dim=karg.pair_embedding_dim,
                           expert_num=karg.expert_num,gate_output_dim=karg.gate_output_dim).to(device)

        trainer(model_water,water_select_expert,model_air,air_select_expert,train_dataloader, model, model_save_path, device, karg.lr, karg.epochs, karg.early_stop_num, karg.verbose, karg.writer_flag)
        water_expert_represents_test = model_water(torch.Tensor(x_test).to(device))[:, water_select_expert, :]
        air_expert_represents_test = model_air(torch.Tensor(x_test).to(device))[:, air_select_expert, :]
        rmse, r2, m, _, mape = predict(model, device, x_test, y_test,water_expert_represents_test,air_expert_represents_test)
        print(f"file:{file_name}, rmse:{rmse}, r2:{r2},mae:{m},mape:{mape}")
        result = pd.DataFrame([rmse + r2 + m + mape],
                              columns=["eads_rmse", "delta_e_rmse", "eb_rmse", "db_rmse", "eads_r2", "delta_e_r2", "eb_r2",
                                       "db_r2", "eads_mae",
                                       "delta_e_mae", "eb_mae", "db_mae", "eads_mape", "delta_e_mape", "eb_mape",
                                       "db_mape"])
        result_path = os.path.join(karg.res_dir, file_name.split(".")[0]+str(k)+"_test.csv")
        create_dir(result_path)
        result.to_csv(result_path, index_label='num')
        
        water_expert_represents_test = model_water(torch.Tensor(transfer_X).to(device))[:, water_select_expert, :]
        air_expert_represents_test = model_air(torch.Tensor(transfer_X).to(device))[:, air_select_expert, :]
        rmse, r2, m, _, mape = predict(model, device, transfer_X, transfer_y,water_expert_represents_test,air_expert_represents_test)
        print(f"file:{file_name}, rmse:{rmse}, r2:{r2},mae:{m},mape:{mape}")
        result = pd.DataFrame([rmse + r2 + m + mape],
                              columns=["eads_rmse", "delta_e_rmse", "eb_rmse", "db_rmse", "eads_r2", "delta_e_r2", "eb_r2",
                                       "db_r2", "eads_mae",
                                       "delta_e_mae", "eb_mae", "db_mae", "eads_mape", "delta_e_mape", "eb_mape",
                                       "db_mape"])
        result_path = os.path.join(karg.res_dir, file_name.split(".")[0]+str(k)+"_transfer.csv")
        create_dir(result_path)
        # 
        result.to_csv(result_path, index_label='num')

Namespace(file_path='data/new_data/process_data/mutilsolvent2.csv', model_save_dir='./p2results_1', res_dir='./p2results_1', water_model_save_path='water2_model_result/waterdi6zhe6.ckpt', air_model_save_path='air2_model_result/air2di9zhe6.ckpt', water_select_expert_config='water2_res_result/waterdi6zhe6_top2.csv', air_select_expert_config='air2_res_result/air2di9zhe6_top2.csv', input_dim=19, represent_dim=100, pair_embedding_dim=5, expert_num=6, gate_output_dim=10, epochs=1, early_stop_num=200, lr=0.0005, verbose=False, writer_flag=False, batch_size=16, test_size=0.2, seed=42, eps='', frozen=False)



====start to train mutilsolvent2.csv====
data/new_data/process_data/mutilsolvent2.csv
dataset num 0
the number of train samples is: 392
the number of test samples is: 98
the number of transfer samples is: 98


100%|██████████| 1/1 [00:03<00:00,  3.05s/it]



Train all epochs.

 Best Model loss_total: 78672.362187, loss1: 0.002382,loss2: 0.015060,loss3: 79.565662,loss4: 11.709820

theta1:Parameter containing:
tensor([0.9875], requires_grad=True),theta2:Parameter containing:
tensor([0.9874], requires_grad=True),theta3:Parameter containing:
tensor([1.0122], requires_grad=True),theta4:Parameter containing:
tensor([1.0121], requires_grad=True)
file:mutilsolvent2.csv, rmse:[0.032675475659511906, 0.03224662495839255, 8.803027967216824, 3.236600248583136], r2:[-0.4467993640440011, -0.7624374781210019, -6015.956659746197, -45227.03873677503],mae:[0.026087597, 0.024604715, 8.802313, 3.236545],mape:[0.37797344, 0.24620257, 0.8499896, 0.78632796]
file:mutilsolvent2.csv, rmse:[0.04116331768411843, 0.11112086344138658, 8.98072233833458, 3.2829881696438514], r2:[-0.09485570826817491, -30.031481250211844, -3222.5439760498048, -17120.551020020186],mae:[0.034213364, 0.08817467, 8.976997, 3.2817318],mape:[0.50587004, 0.9569606, 0.86683434, 0.79485035]
datas

100%|██████████| 1/1 [00:01<00:00,  1.02s/it]



Train all epochs.

 Best Model loss_total: 87239.155313, loss1: 0.086448,loss2: 0.021367,loss3: 88.256545,loss4: 11.128357

theta1:Parameter containing:
tensor([0.9871], requires_grad=True),theta2:Parameter containing:
tensor([0.9874], requires_grad=True),theta3:Parameter containing:
tensor([1.0124], requires_grad=True),theta4:Parameter containing:
tensor([1.0117], requires_grad=True)
file:mutilsolvent2.csv, rmse:[0.053080932224733514, 0.0470809707802269, 9.36216895380185, 3.153534580392523], r2:[-2.804416149935956, -2.770424151594652, -6830.5059635093685, -43027.11805722565],mae:[0.04177314, 0.036250602, 9.361482, 3.1534922],mape:[0.65212274, 0.29335532, 0.9040263, 0.76618165]
file:mutilsolvent2.csv, rmse:[0.05294244610703828, 0.03249892284884742, 9.373438590676507, 3.1654415939840552], r2:[-0.82229190238688, -1.4948933155528001, -3531.9942895862596, -15849.857969112636],mae:[0.043123692, 0.026140125, 9.372158, 3.16531],mape:[0.70356596, 0.23447448, 0.9041359, 0.76674396]
dataset num

...


Train all epochs.

 Best Model loss_total: 95844.790313, loss1: 0.040850,loss2: 0.003580,loss3: 96.910707,loss4: 11.645018

theta1:Parameter containing:
tensor([0.9873], requires_grad=True),theta2:Parameter containing:
tensor([0.9875], requires_grad=True),theta3:Parameter containing:
tensor([1.0121], requires_grad=True),theta4:Parameter containing:
tensor([1.0124], requires_grad=True)
file:mutilsolvent2.csv, rmse:[0.029424128728699043, 0.029197839398162204, 9.488711426281759, 3.329351494679117], r2:[-0.1580283105889626, -0.45135829419056206, -6822.788741242035, -48942.568080143894],mae:[0.024564266, 0.024221132, 9.488008, 3.3293152],mape:[0.28905, 0.20523466, 0.9157442, 0.808812]
file:mutilsolvent2.csv, rmse:[0.04312110436464696, 0.03175464538380813, 9.417335238958508, 3.3402870285600463], r2:[-0.23575796196658616, -0.7485246343267131, -3913.8481710271735, -17454.298710493018],mae:[0.035913777, 0.030283496, 9.416121, 3.3401957],mape:[1.1772085, 0.26071674, 0.9151982, 0.80950904]
datas

100%|██████████| 1/1 [00:00<00:00,  1.01it/s]



Train all epochs.

 Best Model loss_total: 96452.605625, loss1: 0.041073,loss2: 0.008629,loss3: 97.522417,loss4: 13.060181

theta1:Parameter containing:
tensor([0.9873], requires_grad=True),theta2:Parameter containing:
tensor([0.9874], requires_grad=True),theta3:Parameter containing:
tensor([1.0121], requires_grad=True),theta4:Parameter containing:
tensor([1.0113], requires_grad=True)
file:mutilsolvent2.csv, rmse:[0.05390343416848391, 0.027974002950213475, 9.571655677427316, 3.269095151340889], r2:[-3.0688158415394104, -0.33461742950130224, -7034.183687476849, -47054.15869585312],mae:[0.04259162, 0.023337135, 9.570965, 3.26905],mape:[0.44236326, 0.19277537, 0.9234833, 0.79419386]
file:mutilsolvent2.csv, rmse:[0.0661358834120913, 0.03764653589316874, 9.472009481786158, 3.2804392948449888], r2:[-1.9387220764487525, -1.3581044242691944, -4042.7608332042764, -16867.043580809775],mae:[0.058337573, 0.030306492, 9.470953, 3.2803252],mape:[0.82544374, 0.23926811, 0.92235345, 0.7951011]
datase

100%|██████████| 1/1 [00:00<00:00,  1.01it/s]


Train all epochs.

 Best Model loss_total: 87351.810938, loss1: 0.102920,loss2: 0.010716,loss3: 88.366333,loss4: 12.039598

theta1:Parameter containing:
tensor([0.9872], requires_grad=True),theta2:Parameter containing:
tensor([0.9874], requires_grad=True),theta3:Parameter containing:
tensor([1.0124], requires_grad=True),theta4:Parameter containing:
tensor([1.0116], requires_grad=True)
file:mutilsolvent2.csv, rmse:[0.05701325807195492, 0.048326957796150025, 9.363433620214128, 3.235749767978686], r2:[-3.565678051265947, -3.002500655234128, -7077.98899128886, -46100.313130218725],mae:[0.04855852, 0.04047212, 9.362778, 3.2357113],mape:[0.6653318, 0.41960484, 0.9031799, 0.7860896]
file:mutilsolvent2.csv, rmse:[0.061812582319564505, 0.040475661576340485, 9.244515107365691, 3.244781265527864], r2:[-1.5634924423350838, -1.6801095882155361, -3828.7584651376474, -16473.611137366523],mae:[0.05151345, 0.03221558, 9.243343, 3.2446785],mape:[6.539258, 0.30005205, 0.9021518, 0.7865416]
